In [6]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [29]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd

import subprocess
import os
import os.path as osp
from tqdm import tqdm

In [30]:
class MomentsDataset(Dataset):
    def __init__(self, data_path):
        self.data_path = data_path
        self.df = pd.read_csv(self.data_path + 'trainingSet_filtered.csv', header=None)
        self.class_to_index = {}
        actions = [file_name.split('/')[0] for file_name in
                list(self.df[self.df.columns[1]])]

        actions = set(actions)

        for i, action in enumerate(actions):
            self.class_to_index[action] = i


    def get_num_classes(self):
        return len(self.class_to_index)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        print('---- df loc-----')
        print(self.df.iloc[idx])
        print('idx', idx)
        file_loc, label, _, _ = self.df.iloc[idx]
        print('file loc', file_loc)

        full_path = osp.join(self.data_path, 'training', file_loc)
        dir_path = full_path.split('.')[0]

        im_data = []
        images = os.listdir(dir_path)
        for image in images:
            im = cv2.imread(osp.join(dir_path, image))
            resized_image = cv2.resize(im, (128, 128))
            #gray_im = rgb2gray(resized_image)

            im_data.append(resized_image)

        im_data = np.array(im_data)
        im_data = im_data / 255.0
        im_data = im_data[:90]
        if len(im_data) < 90:
            paste_im_data = np.zeros((90, 128, 128, 3))
            paste_im_data[:len(im_data)] = im_data
            paste_im_data[len(im_data):] = im_data[-1]
            im_data = paste_im_data

        index = self.class_to_index[label]

        ret_info = {
                'images': im_data[0],
                'label': index
                }

        return ret_info

In [31]:
train_path = '/hdd/datasets/Moments_in_Time_Mini/training/'

In [33]:
test_path = '/hdd/datasets/Moments_in_Time_Mini/validation/'
for action_folder in os.listdir(test_path):
    action_example_folder = osp.join(test_path, action_folder)

    for action_example in tqdm(os.listdir(action_example_folder)):
        if not action_example.endswith('.mp4'):
            continue
        action_example_name = action_example.split('.')[0]

        write_dir = osp.join(action_example_folder, action_example_name)
        action_example_path = osp.join(action_example_folder, action_example)

        if not osp.exists(write_dir):
            os.makedirs(write_dir)

        run_cmd = 'ffmpeg -loglevel panic -i %s %s' % (action_example_path, write_dir + '/image%d.png')
        os.system(run_cmd)

#open csv
fileIn = open("/hdd/datasets/Moments_in_Time_Mini/validationSet.csv", "r")
#create new csv for writing to 
fileOut = open("/hdd/datasets/Moments_in_Time_Mini/validationSet_filtered.csv", "w")

action_c = 0
action = ""
action_prev = ""

for row in fileIn:
    #get string from row
    split = row.split(".")
    action = split.split("/")
    if action_c == 230:
        if action == action_prev:
            continue
        
    #delimit directory name from string
    directory = split[0]
    #if it exists in /hdd/datasets/Moments_in_Time_Mini/training, write to filtered csv
    if os.path.isdir("/hdd/datasets/Moments_in_Time_Mini/validation/" + directory):
        #write row to trainingSet_filtered.csv
        fileOut.write(row)
    #else, skip over the row
    action_prev = action
    action += 1
    
fileIn.close()
fileOut.close()

  0%|          | 0/50 [00:00<?, ?it/s]


PermissionError: [Errno 13] Permission denied: '/hdd/datasets/Moments_in_Time_Mini/validation/frowning/yt-q2-Xl6CkhDQ_35'

In [34]:


print('data set ')
ds_train = MomentsDataset(train_path)
ds_test = MomentsDataset(test_path)

code_size_train = ds_train.get_num_classes()
print(code_size_train)

code_size_test = ds_test.test.get_num_classes()
print(code_size_test)

#label_count = ds.get_num_classes()

batch_size = 4

print('data loader')
train_loader = DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=1)
test_loader = DataLoader(ds_test, batch_size=batch_size, shuffle=True, num_workers=1)

data set 


FileNotFoundError: File b'/hdd/datasets/Moments_in_Time_Mini/training/trainingSet_filtered.csv' does not exist

In [19]:
SEED = 1
BATCH_SIZE = 128
LOG_INTERVAL = 10
EPOCHS = 10

In [31]:
# connections through the autoencoder bottleneck
# in the pytorch VAE example, this is 20
ZDIMS = 20

In [32]:
# Download or load downloaded MNIST dataset
# shuffle data at every epoch
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True)

# Same for test data
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True)

In [33]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # ENCODER
        # 28 x 28 pixels = 784 input pixels, 400 outputs
        self.fc1 = nn.Linear(784, 400)
        # rectified linear unit layer from 400 to 400
        # max(0, x)
        self.relu = nn.ReLU()
        self.fc21 = nn.Linear(400, ZDIMS)  # mu layer
        self.fc22 = nn.Linear(400, ZDIMS)  # logvariance layer
        # this last layer bottlenecks through ZDIMS connections

        # DECODER
        # from bottleneck to hidden 400
        self.fc3 = nn.Linear(ZDIMS, 400)
        # from hidden 400 to 784 outputs
        self.fc4 = nn.Linear(400, 784)
        self.sigmoid = nn.Sigmoid()

    def encode(self, x: Variable) -> (Variable, Variable):
        """Input vector x -> fully connected 1 -> ReLU -> (fully connected
        21, fully connected 22)

        Parameters
        ----------
        x : [128, 784] matrix; 128 digits of 28x28 pixels each

        Returns
        -------

        (mu, logvar) : ZDIMS mean units one for each latent dimension, ZDIMS
            variance units one for each latent dimension

        """

        # h1 is [128, 400]
        h1 = self.relu(self.fc1(x))  # type: Variable
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu: Variable, logvar: Variable) -> Variable:
        """THE REPARAMETERIZATION IDEA:

        For each training sample (we get 128 batched at a time)

        - take the current learned mu, stddev for each of the ZDIMS
          dimensions and draw a random sample from that distribution
        - the whole network is trained so that these randomly drawn
          samples decode to output that looks like the input
        - which will mean that the std, mu will be learned
          *distributions* that correctly encode the inputs
        - due to the additional KLD term (see loss_function() below)
          the distribution will tend to unit Gaussians

        Parameters
        ----------
        mu : [128, ZDIMS] mean matrix
        logvar : [128, ZDIMS] variance matrix

        Returns
        -------

        During training random sample from the learned ZDIMS-dimensional
        normal distribution; during inference its mean.

        """

        if self.training:
            # multiply log variance with 0.5, then in-place exponent
            # yielding the standard deviation
            std = logvar.mul(0.5).exp_()  # type: Variable
            # - std.data is the [128,ZDIMS] tensor that is wrapped by std
            # - so eps is [128,ZDIMS] with all elements drawn from a mean 0
            #   and stddev 1 normal distribution that is 128 samples
            #   of random ZDIMS-float vectors
            eps = Variable(std.data.new(std.size()).normal_())
            # - sample from a normal distribution with standard
            #   deviation = std and mean = mu by multiplying mean 0
            #   stddev 1 sample with desired std and mu, see
            #   https://stats.stackexchange.com/a/16338
            # - so we have 128 sets (the batch) of random ZDIMS-float
            #   vectors sampled from normal distribution with learned
            #   std and mu for the current input
            return eps.mul(std).add_(mu)

        else:
            # During inference, we simply spit out the mean of the
            # learned distribution for the current input.  We could
            # use a random sample from the distribution, but mu of
            # course has the highest probability.
            return mu

    def decode(self, z: Variable) -> Variable:
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x: Variable) -> (Variable, Variable, Variable):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [34]:
model = VAE()

In [46]:
def loss_function(recon_x, x, mu, logvar) -> Variable:
    # how well do input x and output recon_x agree?
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784))

    # KLD is Kullback–Leibler divergence -- how much does one learned
    # distribution deviate from another, in this specific case the
    # learned distribution from the unit Gaussian

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # - D_{KL} = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    # note the negative D_{KL} in appendix B of the paper
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    # Normalise by same number of elements as in reconstruction
    KLD /= BATCH_SIZE * 784

    # BCE tries to make our reconstruction as accurate as possible
    # KLD tries to push the distributions as close as possible to unit Gaussian
    return BCE + KLD

In [47]:
# Dr Diederik Kingma: as if VAEs weren't enough, he also gave us Adam!
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [48]:
def train(epoch):
    # toggle model to train mode
    model.train()
    train_loss = 0
    # in the case of MNIST, len(train_loader.dataset) is 60000
    # each `data` is of BATCH_SIZE samples and has shape [128, 1, 28, 28]
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        optimizer.zero_grad()

        # push whole batch of data through VAE.forward() to get recon_loss
        recon_batch, mu, logvar = model(data)
        # calculate scalar loss
        loss = loss_function(recon_batch, data, mu, logvar)
        # calculate the gradient of the loss w.r.t. the graph leaves
        # i.e. input variables -- by the power of pytorch!
        loss.backward()
        train_loss += loss.item() #loss.data[0]
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data))) #loss.data[0]

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [53]:
def test(epoch):
    
    # toggle model to test / inference mode
    model.eval()
    test_loss = 0

    # each data is of BATCH_SIZE (default 128) samples
    for i, (data, _) in enumerate(test_loader):

        # we're only going to infer, so no autograd at all required: volatile=True
        data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        test_loss += loss_function(recon_batch, data, mu, logvar).item() #data[0]
        if i == 0:
            n = min(data.size(0), 8)
          # for the first 128 batch of the epoch, show the first 8 input digits
          # with right below them the reconstructed output digits
            comparison = torch.cat([data[:n],
                                  recon_batch.view(BATCH_SIZE, 1, 28, 28)[:n]])
            save_image(comparison.data.cpu(),
                  'results_vae/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [54]:
for epoch in range(1, EPOCHS + 1):
    train(epoch)
    test(epoch)

    # 64 sets of random ZDIMS-float vectors, i.e. 64 locations / MNIST
    # digits in latent space
    sample = Variable(torch.randn(64, ZDIMS))
    sample = model.decode(sample).cpu()

    # save out as an 8x8 matrix of MNIST digits
    # this will give you a visual idea of how well latent space can generate things
    # that look like digits
    save_image(sample.data.view(64, 1, 28, 28),
               'results_vae/sample_' + str(epoch) + '.png')

eepoch 1
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.001052
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.001083
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.001018
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.001023
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.001042
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.001034
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.001012
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.001028
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.001021
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.001080
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.001055
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.001065
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.001036
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.001043
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.001051
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.001015
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.001060
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.001080
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.001025
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.001015
Train E

/opt/conda/envs/pt/lib/python3.5/site-packages/ipykernel_launcher.py:11: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # This is added back by InteractiveShellApp.init_path()


====> Test set loss: 0.0010
eepoch 2
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.001069
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.001047
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.001052
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.001011
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.001066
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.001073
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.001063
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.001036
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.001037
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.001057
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.001072
Train Epoch: 2 [14080/60000 (23%)]	Loss: 0.001060
Train Epoch: 2 [15360/60000 (26%)]	Loss: 0.001045
Train Epoch: 2 [16640/60000 (28%)]	Loss: 0.001025
Train Epoch: 2 [17920/60000 (30%)]	Loss: 0.001042
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.001047
Train Epoch: 2 [20480/60000 (34%)]	Loss: 0.001015
Train Epoch: 2 [21760/60000 (36%)]	Loss: 0.001021
Train Epoch: 2 [23040/60000 (38%)]	Loss: 0.001021
Train Epoch: 2 [24320/60000 (

Train Epoch: 5 [25600/60000 (43%)]	Loss: 0.000997
Train Epoch: 5 [26880/60000 (45%)]	Loss: 0.000999
Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.001074
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.001051
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.001076
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.001040
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.001026
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.001043
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.001037
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.001069
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.001067
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.001034
Train Epoch: 5 [40960/60000 (68%)]	Loss: 0.001070
Train Epoch: 5 [42240/60000 (70%)]	Loss: 0.001002
Train Epoch: 5 [43520/60000 (72%)]	Loss: 0.000994
Train Epoch: 5 [44800/60000 (75%)]	Loss: 0.001042
Train Epoch: 5 [46080/60000 (77%)]	Loss: 0.001024
Train Epoch: 5 [47360/60000 (79%)]	Loss: 0.001039
Train Epoch: 5 [48640/60000 (81%)]	Loss: 0.001033
Train Epoch: 5 [49920/60000 (83%)]	Loss: 0.001046


Train Epoch: 8 [51200/60000 (85%)]	Loss: 0.001032
Train Epoch: 8 [52480/60000 (87%)]	Loss: 0.001037
Train Epoch: 8 [53760/60000 (90%)]	Loss: 0.001064
Train Epoch: 8 [55040/60000 (92%)]	Loss: 0.001031
Train Epoch: 8 [56320/60000 (94%)]	Loss: 0.001024
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.001024
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.001041
====> Epoch: 8 Average loss: 0.0010
====> Test set loss: 0.0009
eepoch 9
Train Epoch: 9 [0/60000 (0%)]	Loss: 0.001055
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.001041
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.001055
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.001016
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.001027
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.001061
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.001014
Train Epoch: 9 [8960/60000 (15%)]	Loss: 0.001021
Train Epoch: 9 [10240/60000 (17%)]	Loss: 0.000979
Train Epoch: 9 [11520/60000 (19%)]	Loss: 0.000994
Train Epoch: 9 [12800/60000 (21%)]	Loss: 0.001043
Train Epoch: 9 [14080/60000 (23%)]	Loss: 0.